<a href="https://colab.research.google.com/github/Python-Neiva/risk-factors-covid19/blob/main/risk_factor_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Request

In [2]:
import requests

csv_url = "https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv?departamento_nom=AMAZONAS&accessType=DOWNLOAD&api_foundry=true"
#csv_url = "./content/sample_data/sample_downloaded.csv"
root_path = "/content/drive/MyDrive/Salisbury/data/covid_downloaded.csv"
def downloadDataCsv(csv_url,root_path):
  req = requests.get(csv_url)
  url_content = req.content
  try:
      csv_file = open(root_path, 'wb')

      csv_file.write(url_content)

      csv_file.close()
      print("I finished to write csv")

  except:
      print("something was wrong")

#downloadDataCsv(csv_url,root_path)

#Model

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import math

# import os
# !pip install sodapy -q
# !pip install socrata-py -q

# from sodapy import Socrata
# from socrata.authorization import Authorization


In [4]:
def requestColombianData(limit = 1000):
  client = Socrata(
                  "www.datos.gov.co",
                  "idLwSviPi5dGMTd7GfIfW3CLU",
                  #username="sebastian_romerola@fet.edu.co",
                  #password="Sebastian#2018",
                  timeout=300
                  )
  results = client.get("gt2j-8ykr",
                      #  ciudad_municipio_nom="Neiva",
                      departamento_nom="ANTIOQUIA",
                      limit=limit
                      )
  
  return pd.DataFrame.from_records(results)


def getCsvDataSet(path='/content/drive/MyDrive/Salisbury/data/covid_downloaded.csv'):
  return pd.read_csv(r''+path, low_memory=False)

In [5]:
#dataset from csv
df = getCsvDataSet()

#dataset from API
# df = None
# df = requestColombianData(90000)

In [6]:
df.head(20)

,fecha reporte web,ID de caso,Fecha de notificación,Código DIVIPOLA departamento,Nombre departamento,Código DIVIPOLA municipio,Nombre municipio,Edad,Unidad de medida de edad,Sexo,Tipo de contagio,Ubicación del caso,Estado,Código ISO del país,Nombre del país,Recuperado,Fecha de inicio de síntomas,Fecha de muerte,Fecha de diagnóstico,Fecha de recuperación,Tipo de recuperación,Pertenencia étnica,Nombre del grupo étnico
0,6/3/2020 0:00:00,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,Importado,Casa,Leve,380.0,ITALIA,Recuperado,27/2/2020 0:00:00,NaN,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6.0,NaN
1,9/3/2020 0:00:00,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,4/3/2020 0:00:00,NaN,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5.0,NaN
2,9/3/2020 0:00:00,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,29/2/2020 0:00:00,NaN,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6.0,NaN
3,11/3/2020 0:00:00,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6.0,NaN
4,11/3/2020 0:00:00,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,8/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6.0,NaN
5,11/3/2020 0:00:00,6,10/3/2020 0:00:00,5,ANTIOQUIA,5360,ITAGUI,27,1,F,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6.0,NaN
6,11/3/2020 0:00:00,7,8/3/2020 0:00:00,13001,CARTAGENA,13001,CARTAGENA,85,1,F,Importado,Casa,Leve,840.0,ESTADOS UNIDOS DE AMÉRICA,Recuperado,2/3/2020 0:00:00,NaN,11/3/2020 0:00:00,17/3/2020 0:00:00,PCR,6.0,NaN
7,11/3/2020 0:00:00,8,9/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,22,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6.0,NaN
8,11/3/2020 0:00:00,9,8/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,28,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,7/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6.0,NaN
9,12/3/2020 0:00:00,10,12/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,36,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,6/3/2020 0:00:00,NaN,12/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6.0,NaN


In [7]:
print(df.columns)
english_columns = list()
english_columns = [
  'web_date_report',
  'case_id',
  'notification_date',
  'davipola_departament_code',
  'department_name',
  'davipola_municipality_code',
  'municipality_name',
  'age',
  'age_measurement_unit',
  'sex',
  'contagion_type',
  'case_location',
  'state',
  'iso_country_code',
  'country_name',
  'recuperated',
  'beginning_symptoms_date',
  'date_of_death',
  'date_of_diagnosis',
  'date_of_recovery',
  'recovery_type',
  'ethnicity',
  'ethnic_group_name',
]

df.columns = english_columns
print("-----")
print(df.columns)

Index(['fecha reporte web', 'ID de caso', 'Fecha de notificación',
       'Código DIVIPOLA departamento', 'Nombre departamento',
       'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad',
       'Unidad de medida de edad', 'Sexo', 'Tipo de contagio',
       'Ubicación del caso', 'Estado', 'Código ISO del país',
       'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas',
       'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación',
       'Tipo de recuperación', 'Pertenencia étnica',
       'Nombre del grupo étnico'],
      dtype='object')
-----
Index(['web_date_report', 'case_id', 'notification_date',
       'davipola_departament_code', 'department_name',
       'davipola_municipality_code', 'municipality_name', 'age',
       'age_measurement_unit', 'sex', 'contagion_type', 'case_location',
       'state', 'iso_country_code', 'country_name', 'recuperated',
       'beginning_symptoms_date', 'date_of_death', 'date_of_diagnosis',
       'date_of_recovery', 're

In [8]:
df.head()

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name
0,6/3/2020 0:00:00,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,Importado,Casa,Leve,380.0,ITALIA,Recuperado,27/2/2020 0:00:00,NaN,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6.0,NaN
1,9/3/2020 0:00:00,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,4/3/2020 0:00:00,NaN,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5.0,NaN
2,9/3/2020 0:00:00,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,Importado,Casa,Leve,724.0,ESPAÑA,Recuperado,29/2/2020 0:00:00,NaN,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6.0,NaN
3,11/3/2020 0:00:00,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6.0,NaN
4,11/3/2020 0:00:00,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,8/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6.0,NaN


In [9]:
df.astype = {
  "web_date_report": np.str,
  "case_id": np.str, #ordinal
  "notification_date": np.str,
  "davipola_departament_code": np.int, #categorical
  "department_name": np.str, #categorical
  "davipola_municipality_code": np.str, #categorical
  "municipality_name": np.str, #categorical
  "age": np.int,
  "age_measurement_unit": np.int,
  "sex": np.str, #nominal
  "contagion_type": np.str, #categorical
  "case_location": np.str, #categorical
  "state": np.str, #ordinal
  "iso_country_code": np.int, #categorical
  "country_name": np.str, #categorical
  "recuperated": np.str, ##categorical
  "beginning_symptoms_date": np.str, #
  "date_of_diagnosis": np.str,
  "date_of_recovery": np.str,
  "recovery_type": np.str, #categorica
  "ethnicity": np.int, #nominal
  "ethnic_group_name": np.str, #nominal
  "date_of_death": np.str,
}

#change column type in df

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4530610 entries, 0 to 4530609
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   web_date_report             object 
 1   case_id                     int64  
 2   notification_date           object 
 3   davipola_departament_code   int64  
 4   department_name             object 
 5   davipola_municipality_code  int64  
 6   municipality_name           object 
 7   age                         int64  
 8   age_measurement_unit        int64  
 9   sex                         object 
 10  contagion_type              object 
 11  case_location               object 
 12  state                       object 
 13  iso_country_code            float64
 14  country_name                object 
 15  recuperated                 object 
 16  beginning_symptoms_date     object 
 17  date_of_death               object 
 18  date_of_diagnosis           object 
 19  date_of_recovery     

In [11]:
df.describe()

,case_id,davipola_departament_code,davipola_municipality_code,age,age_measurement_unit,iso_country_code,ethnicity
count,4.530610e+06,4.530610e+06,4.530610e+06,4.530610e+06,4.530610e+06,3057.000000,4.471622e+06
mean,2.265345e+06,1.302291e+03,2.804815e+04,3.969038e+01,1.003440e+00,597.879293,5.907317e+00
std,1.307875e+06,5.853340e+03,2.598089e+04,1.788452e+01,6.372697e-02,285.800759,6.064583e-01
min,1.000000e+00,5.000000e+00,5.001000e+03,1.000000e+00,1.000000e+00,4.000000,1.000000e+00
25%,1.132693e+06,1.100000e+01,1.100100e+04,2.700000e+01,1.000000e+00,380.000000,6.000000e+00
50%,2.265346e+06,1.700000e+01,1.100100e+04,3.700000e+01,1.000000e+00,724.000000,6.000000e+00
75%,3.397998e+06,6.800000e+01,5.000100e+04,5.200000e+01,1.000000e+00,840.000000,6.000000e+00
max,4.530650e+06,4.700100e+04,9.977300e+04,1.140000e+02,3.000000e+00,1724.000000,6.000000e+00


In [12]:
np.shape(df)

(4530610, 23)

In [13]:
df.columns

Index(['web_date_report', 'case_id', 'notification_date',
       'davipola_departament_code', 'department_name',
       'davipola_municipality_code', 'municipality_name', 'age',
       'age_measurement_unit', 'sex', 'contagion_type', 'case_location',
       'state', 'iso_country_code', 'country_name', 'recuperated',
       'beginning_symptoms_date', 'date_of_death', 'date_of_diagnosis',
       'date_of_recovery', 'recovery_type', 'ethnicity', 'ethnic_group_name'],
      dtype='object')

uppercase to each str row

In [14]:
df.select_dtypes(include='object')

,web_date_report,notification_date,department_name,municipality_name,sex,contagion_type,case_location,state,country_name,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,date_of_recovery,recovery_type,ethnic_group_name
0,6/3/2020 0:00:00,2/3/2020 0:00:00,BOGOTA,BOGOTA,F,Importado,Casa,Leve,ITALIA,Recuperado,27/2/2020 0:00:00,NaN,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,NaN
1,9/3/2020 0:00:00,6/3/2020 0:00:00,VALLE,BUGA,M,Importado,Casa,Leve,ESPAÑA,Recuperado,4/3/2020 0:00:00,NaN,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,NaN
2,9/3/2020 0:00:00,7/3/2020 0:00:00,ANTIOQUIA,MEDELLIN,F,Importado,Casa,Leve,ESPAÑA,Recuperado,29/2/2020 0:00:00,NaN,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,NaN
3,11/3/2020 0:00:00,9/3/2020 0:00:00,ANTIOQUIA,MEDELLIN,M,Relacionado,Casa,Leve,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,NaN
4,11/3/2020 0:00:00,9/3/2020 0:00:00,ANTIOQUIA,MEDELLIN,M,Relacionado,Casa,Leve,NaN,Recuperado,8/3/2020 0:00:00,NaN,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4530605,12/7/2021 0:00:00,29/6/2021 0:00:00,NORTE SANTANDER,CUCUTA,M,En estudio,Casa,Leve,NaN,Activo,25/6/2021 0:00:00,NaN,10/7/2021 0:00:00,NaN,NaN,NaN
4530606,12/7/2021 0:00:00,29/6/2021 0:00:00,BOLIVAR,SAN JUAN NEPOMUCENO,M,En estudio,Casa,Leve,NaN,Activo,25/6/2021 0:00:00,NaN,10/7/2021 0:00:00,NaN,NaN,NaN
4530607,12/7/2021 0:00:00,29/6/2021 0:00:00,NORTE SANTANDER,VILLA DEL ROSARIO,M,En estudio,Casa,Leve,NaN,Activo,25/6/2021 0:00:00,NaN,10/7/2021 0:00:00,NaN,NaN,NaN
4530608,12/7/2021 0:00:00,29/6/2021 0:00:00,NORTE SANTANDER,CUCUTA,M,En estudio,Casa,Leve,NaN,Activo,25/6/2021 0:00:00,NaN,10/7/2021 0:00:00,NaN,NaN,NaN


verifying nulls


```
print(df.isnull().values.any())
```
[source](https://towardsdatascience.com/a-complete-guide-to-principal-component-analysis-pca-in-machine-learning-664f34fc3e5a)

After use this function, the result of the code is false.


In [15]:
print(df.isnull().values.any())

True


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4530610 entries, 0 to 4530609
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   web_date_report             object 
 1   case_id                     int64  
 2   notification_date           object 
 3   davipola_departament_code   int64  
 4   department_name             object 
 5   davipola_municipality_code  int64  
 6   municipality_name           object 
 7   age                         int64  
 8   age_measurement_unit        int64  
 9   sex                         object 
 10  contagion_type              object 
 11  case_location               object 
 12  state                       object 
 13  iso_country_code            float64
 14  country_name                object 
 15  recuperated                 object 
 16  beginning_symptoms_date     object 
 17  date_of_death               object 
 18  date_of_diagnosis           object 
 19  date_of_recovery     

Similarity mesurement (radio basics funciotn)
1.  
*   ML / rbf -| knn graph / 
*   train to know about who would most highly survive
*   KNN graph (find the most near neibough
2.  
*   svm (support vector machine)





Adjency matrix. to build a graph after apply rbf and knn aproach?



In [17]:
df["beginning_symptoms_date"] = pd.to_datetime(df["beginning_symptoms_date"],format='%d/%m/%Y %H:%M:%S')
df["date_of_death"] = pd.to_datetime(df["date_of_death"],format='%d/%m/%Y %H:%M:%S')
df["web_date_report"] = pd.to_datetime(df["web_date_report"],format='%d/%m/%Y %H:%M:%S')
df["date_of_recovery"] = pd.to_datetime(df["date_of_recovery"],format='%d/%m/%Y %H:%M:%S')
df["date_of_diagnosis"] = pd.to_datetime(df["date_of_diagnosis"],format='%d/%m/%Y %H:%M:%S')
df["notification_date"] = pd.to_datetime(df["notification_date"],format='%d/%m/%Y %H:%M:%S')

Does is first greather than second?

In [18]:
df["beginning_symptoms_date"][0] > df["beginning_symptoms_date"][1]

False

This let find categories, regardless upper or lower case.

In [19]:
for i in df.select_dtypes(include='object'):
  df[i] = df[i].str.upper()

Analisis de componentes principales (APC)

quitar lo que se puede.



desviación estandar de  para analizar causas de muerte

*italicized text*.fit

se realiza conversión a todos los que tengan

In [20]:
#////////////////////////////////////////////////////////#
#                                                        /
#      Binarize Sex column values to avoid errors        /
#                                                        /
#////////////////////////////////////////////////////////#
from sklearn.preprocessing import Binarizer

#/-////////////////////////////////////////////////////////#
#                                                         /
#              average of recovery time per genre         /
#                                                         /
#/////////////////////////////////////////////////////////#

#////////////////////////////////////////////////////////#
#                                                        /
#     Capitalize Sex column values to avoid errors       /
#                                                        /
#////////////////////////////////////////////////////////#

#////////////////////////////////////////////////////////#
#                                                        /
#  substract betweeen two dates and store it the answer  /
#                                                        /
#////////////////////////////////////////////////////////#

def get_duration_of_illness(x:any):
  time = 0
  z = list()
  count = 0
  for e in x.index:
    count += 1
    if x.date_of_recovery[e]:
      #time delta - .dt.days to integer
      z.append(x.date_of_recovery[e] - x.beginning_symptoms_date[e])
    elif x.date_of_death[e]:
      z.append(x.date_of_death[e] - x.beginning_symptoms_date[e])
    elif x.date_of_death[e] == 0 and x.date_of_recovery[e] == 0:
      print("Unknown")
      x.loc[e, "illness_time"] = None

  print("The total of patients is: "+ str(count))

  return x
  
#array_dates_new_dates = get_duration_of_illness(df)

Replacing


In [21]:
#df.boxplot('age','contagion_type',rot = 0,figsize=(5,6))

# Data Cleaning
Label or one hot enconding or replacing



In [22]:
df.dtypes.value_counts()

object            10
datetime64[ns]     6
int64              5
float64            2
dtype: int64

In [23]:
df['sex'].describe()

count     4530610
unique          2
top             F
freq      2372713
Name: sex, dtype: object

*  Fill empty spaces with 0.
*  Determine unneccessary feateres (columns).
*  Delete departament name && municipality name.
*  Binarize sex column.
*  Delete country name.
*  Normalize recovery type.
*  Write documentation.


## Fill empty spaces with 0.


In [24]:
df.isnull().sum()

web_date_report                     0
case_id                             0
notification_date                   0
davipola_departament_code           0
department_name                     0
davipola_municipality_code          0
municipality_name                   0
age                                 0
age_measurement_unit                0
sex                                 0
contagion_type                      0
case_location                   15212
state                           15212
iso_country_code              4527553
country_name                  4527545
recuperated                     12806
beginning_symptoms_date        516910
date_of_death                 4402058
date_of_diagnosis                4106
date_of_recovery               254145
recovery_type                  254145
ethnicity                       58988
ethnic_group_name             4467698
dtype: int64

In [25]:
df.head(10)

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name
0,2020-03-06,1,2020-03-02,11,BOGOTA,11001,BOGOTA,19,1,F,IMPORTADO,CASA,LEVE,380.0,ITALIA,RECUPERADO,2020-02-27,NaT,2020-03-06,2020-03-13,PCR,6.0,NaN
1,2020-03-09,2,2020-03-06,76,VALLE,76111,BUGA,34,1,M,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-03-04,NaT,2020-03-09,2020-03-19,PCR,5.0,NaN
2,2020-03-09,3,2020-03-07,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-02-29,NaT,2020-03-09,2020-03-15,PCR,6.0,NaN
3,2020-03-11,4,2020-03-09,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,2020-03-06,NaT,2020-03-11,2020-03-26,PCR,6.0,NaN
4,2020-03-11,5,2020-03-09,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,2020-03-08,NaT,2020-03-11,2020-03-23,PCR,6.0,NaN
5,2020-03-11,6,2020-03-10,5,ANTIOQUIA,5360,ITAGUI,27,1,F,RELACIONADO,CASA,LEVE,NaN,NaN,RECUPERADO,2020-03-06,NaT,2020-03-11,2020-03-26,PCR,6.0,NaN
6,2020-03-11,7,2020-03-08,13001,CARTAGENA,13001,CARTAGENA,85,1,F,IMPORTADO,CASA,LEVE,840.0,ESTADOS UNIDOS DE AMÉRICA,RECUPERADO,2020-03-02,NaT,2020-03-11,2020-03-17,PCR,6.0,NaN
7,2020-03-11,8,2020-03-09,11,BOGOTA,11001,BOGOTA,22,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-03-06,NaT,2020-03-11,2020-03-21,PCR,6.0,NaN
8,2020-03-11,9,2020-03-08,11,BOGOTA,11001,BOGOTA,28,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-03-07,NaT,2020-03-11,2020-03-23,PCR,6.0,NaN
9,2020-03-12,10,2020-03-12,11,BOGOTA,11001,BOGOTA,36,1,F,IMPORTADO,CASA,LEVE,724.0,ESPAÑA,RECUPERADO,2020-03-06,NaT,2020-03-12,2020-03-21,PCR,6.0,NaN


#Data Transformation

In [26]:
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import StandardScaler

import xgboost
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from math import sqrt

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler #,OrdinalEncoder

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

##Categorical Variables





> Information about: Dinesh Yadav, https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd.



Posible ways to replace
[Pandas Documentation 
](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html)
```
df.replace({'A': {0: 100, 4: 400}})
Result:
        A  B  C
0  100  5  a
1    1  6  b
2    2  7  c
3    3  8  d
4  400  9  e
```



standarization

## Data import for test


In [28]:
#storage test data
path = '/content/drive/MyDrive/Salisbury/data/covid_downloaded_1000.csv'
train_csv_url = 'https://www.datos.gov.co/resource/gt2j-8ykr.csv'
# downloadDataCsv(train_csv_url,path)

In [29]:
#reading storaged test data
df_to_test = getCsvDataSet(path)
df_to_test.columns = english_columns
df_to_test = df_to_test.iloc[:, lambda df_to_test: [7,9,10,11,12,13,15,16,17,18,20,21]].copy()

In [30]:
print(df_to_test.info)
df_to_test.head()

<bound method DataFrame.info of      age sex contagion_type  ...  date_of_diagnosis recovery_type  ethnicity
0     19   F      Importado  ...   6/3/2020 0:00:00           PCR          6
1     34   M      Importado  ...   9/3/2020 0:00:00           PCR          5
2     50   F      Importado  ...   9/3/2020 0:00:00           PCR          6
3     55   M    Relacionado  ...  11/3/2020 0:00:00           PCR          6
4     25   M    Relacionado  ...  11/3/2020 0:00:00           PCR          6
..   ...  ..            ...  ...                ...           ...        ...
995   30   F    Comunitaria  ...   1/4/2020 0:00:00           PCR          6
996   44   F    Comunitaria  ...   1/4/2020 0:00:00           PCR          6
997   17   F    Comunitaria  ...   1/4/2020 0:00:00           PCR          6
998   39   F    Comunitaria  ...   1/4/2020 0:00:00           PCR          6
999   30   F    Comunitaria  ...   1/4/2020 0:00:00           PCR          6

[1000 rows x 12 columns]>


,age,sex,contagion_type,case_location,state,iso_country_code,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,recovery_type,ethnicity
0,19,F,Importado,Casa,Leve,380.0,Recuperado,27/2/2020 0:00:00,NaN,6/3/2020 0:00:00,PCR,6
1,34,M,Importado,Casa,Leve,724.0,Recuperado,4/3/2020 0:00:00,NaN,9/3/2020 0:00:00,PCR,5
2,50,F,Importado,Casa,Leve,724.0,Recuperado,29/2/2020 0:00:00,NaN,9/3/2020 0:00:00,PCR,6
3,55,M,Relacionado,Casa,Leve,NaN,Recuperado,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,PCR,6
4,25,M,Relacionado,Casa,Leve,NaN,Recuperado,8/3/2020 0:00:00,NaN,11/3/2020 0:00:00,PCR,6


In [31]:
for i in df_to_test.select_dtypes(include='object'):
  df_to_test[i] = df_to_test[i].str.upper()

In [32]:
df_to_test.head()

,age,sex,contagion_type,case_location,state,iso_country_code,recuperated,beginning_symptoms_date,date_of_death,date_of_diagnosis,recovery_type,ethnicity
0,19,F,IMPORTADO,CASA,LEVE,380.0,RECUPERADO,27/2/2020 0:00:00,NaN,6/3/2020 0:00:00,PCR,6
1,34,M,IMPORTADO,CASA,LEVE,724.0,RECUPERADO,4/3/2020 0:00:00,NaN,9/3/2020 0:00:00,PCR,5
2,50,F,IMPORTADO,CASA,LEVE,724.0,RECUPERADO,29/2/2020 0:00:00,NaN,9/3/2020 0:00:00,PCR,6
3,55,M,RELACIONADO,CASA,LEVE,NaN,RECUPERADO,6/3/2020 0:00:00,NaN,11/3/2020 0:00:00,PCR,6
4,25,M,RELACIONADO,CASA,LEVE,NaN,RECUPERADO,8/3/2020 0:00:00,NaN,11/3/2020 0:00:00,PCR,6


##Train_data

In [33]:
df_to_train = df.iloc[:, lambda df: [7,9,10,11,12,13,15,
                                     #16,17,18,
                                     20,21]].copy()
df_to_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4530610 entries, 0 to 4530609
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   age               int64  
 1   sex               object 
 2   contagion_type    object 
 3   case_location     object 
 4   state             object 
 5   iso_country_code  float64
 6   recuperated       object 
 7   recovery_type     object 
 8   ethnicity         float64
dtypes: float64(2), int64(1), object(6)
memory usage: 311.1+ MB


##Preprocesing


**Take** in mind

*   https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd

*   https://plotly.com/python/knn-classification/




##functions

issue: It is not posible to work with NaN, so, it is neccessary to replace NaN values per something else, in order to be able to apply fit_transform to this ordinal variable.

In [34]:
for i in df_to_train.columns:
  print(i)
  df_to_train[i] = df_to_train[i].fillna(0)

age
sex
contagion_type
case_location
state
iso_country_code
recuperated
recovery_type
ethnicity


In [35]:
df_to_train.isnull().sum()

age                 0
sex                 0
contagion_type      0
case_location       0
state               0
iso_country_code    0
recuperated         0
recovery_type       0
ethnicity           0
dtype: int64

In [36]:
# # day
# df_to_train['date_of_death_day'] = df_to_train['date_of_death'].dt.day
# # month
# df_to_train['date_of_death_month'] = df_to_train['date_of_death'].dt.month
# # year
# df_to_train['date_of_death_year'] = df_to_train['date_of_death'].dt.year

KeyError: ignored

In [ ]:
#(df['state'].unique())
state_categories = [ 0,'LEVE','MODERADO', 'GRAVE', 'FALLECIDO']
print(state_categories)

In [ ]:
oe = OrdinalEncoder(categories = [state_categories])
ohe = OneHotEncoder(sparse = False)

In [ ]:
numeric_features = ['age']
categorical_features = ['sex','contagion_type','case_location','state','recuperated','recovery_type','ethnicity']

##applying data preprocesing

In [38]:
X = df_to_train.drop('recovery_type', axis = 1)
y = df_to_train['recovery_type']

In [100]:
column_transform = make_column_transformer(
    (ohe, ['sex','contagion_type','case_location','recuperated']), 
    (oe, ['state'])
    )

#['sex','contagion_type','case_location','state','recuperated','recovery_type']

In [ ]:
column_transform.fit_transform(X)

##Column Transformer with Mixed Types

In [40]:
numeric_features = ['age']
categorical_features = ['sex','contagion_type','case_location','state','recovery_type','ethnicity']

X = df_to_train.drop('recuperated', axis = 1)
y = df_to_train['recuperated']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=0)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

NameError: ignored

In [47]:
X


,age,sex,contagion_type,case_location,state,iso_country_code,recovery_type,ethnicity
0,19,F,IMPORTADO,CASA,LEVE,380.0,PCR,6.0
1,34,M,IMPORTADO,CASA,LEVE,724.0,PCR,5.0
2,50,F,IMPORTADO,CASA,LEVE,724.0,PCR,6.0
3,55,M,RELACIONADO,CASA,LEVE,0.0,PCR,6.0
4,25,M,RELACIONADO,CASA,LEVE,0.0,PCR,6.0
...,...,...,...,...,...,...,...,...
4530605,46,M,EN ESTUDIO,CASA,LEVE,0.0,0,0.0
4530606,30,M,EN ESTUDIO,CASA,LEVE,0.0,0,0.0
4530607,11,M,EN ESTUDIO,CASA,LEVE,0.0,0,0.0
4530608,29,M,EN ESTUDIO,CASA,LEVE,0.0,0,0.0


##Machine Learning Process

In [ ]:
df_to_train
merge_data = df_to_train.append(df_to_test)
test_count = len(df_to_test)
count = len(merge_data)-test_count
X_cat = merge_data.copy()
X_cat = merge_data.select_dtypes(include=['object'])
X_enc = X_cat.copy()

In [ ]:
X_cat = df_to_train.select_dtypes(include=['object'])
X_enc = X_cat.copy()

In [ ]:
#ONEHOT ENCODING BLOCK
X_enc = pd.get_dummies(X_enc, columns=columns_to_hot_encoding)
df_to_train = df_to_train.drop(columns_to_hot_encoding,axis=1)

In [ ]:
X_enc

,state,beginning_symptoms_date,date_of_death,date_of_diagnosis,sex_F,sex_M,contagion_type_COMUNITARIA,contagion_type_EN ESTUDIO,contagion_type_IMPORTADO,contagion_type_RELACIONADO,case_location_CASA,case_location_FALLECIDO,case_location_HOSPITAL,case_location_HOSPITAL UCI,recuperated_ACTIVO,recuperated_FALLECIDO,recuperated_RECUPERADO,recovery_type_PCR,recovery_type_TIEMPO
0,LEVE,2020-02-27 00:00:00,NaT,2020-03-06 00:00:00,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0
1,LEVE,2020-03-04 00:00:00,NaT,2020-03-09 00:00:00,0,1,0,0,1,0,1,0,0,0,0,0,1,1,0
2,LEVE,2020-02-29 00:00:00,NaT,2020-03-09 00:00:00,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0
3,LEVE,2020-03-06 00:00:00,NaT,2020-03-11 00:00:00,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0
4,LEVE,2020-03-08 00:00:00,NaT,2020-03-11 00:00:00,0,1,0,0,0,1,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,LEVE,19/3/2020 0:00:00,NaN,1/4/2020 0:00:00,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0
996,LEVE,21/3/2020 0:00:00,NaN,1/4/2020 0:00:00,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0
997,LEVE,17/3/2020 0:00:00,NaN,1/4/2020 0:00:00,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0
998,LEVE,18/3/2020 0:00:00,NaN,1/4/2020 0:00:00,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0


In [ ]:
#ONEHOT ENCODING BLOCK#X_enc = pd.get_dummies(X_enc, columns=['sex','region','smoker'])

#mergedata = mergedata.drop(['sex','region','smoker'],axis=1)

##Pipeline
[Using scikit-learn](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html)

#API

##Compromiso

1. Servicio que colecta la data por departamento y la guarda en la base de datos
    * Eliminar registros por departamento y volver a consultarlos y almacenarlos
2. Servicio que recibe por parametro el id del departamento 
3. Servicio que recupera la info por municipio.


##Commitment

1. Service which collect the data from Colombia according to each departament (state)
    * Delete records per departament and request it again
2. Service which get by parameter the departament_id and return the information
3. Service get municipality info


#Documentation


request
[location and count](https://datos.gov.co/resource/gt2j-8ykr.json?$select=%20count(id_de_caso),ubicacion&$group=ubicacion)

Json
>```
count_id_de_caso	"11015"
ubicacion	"casa"
count_id_de_caso	"4028443"
ubicacion	"Casa"
count_id_de_caso	"7"
ubicacion	"CASA"
count_id_de_caso	"105326"
ubicacion	"Fallecido"
count_id_de_caso	"22865"
ubicacion	"Hospital"
count_id_de_caso	"4963"
ubicacion	"Hospital UCI"
count_id_de_caso	"14575"
ubicacion	"N/A"
```



Patient State [request](https://datos.gov.co/resource/gt2j-8ykr.json?$select=%20estado&$group=estado)


***Patient state *** 

>```
[{"estado":"Fallecido"}
,{"estado":"Grave"}
,{"estado":"leve"}
,{"estado":"Leve"}
,{"estado":"LEVE"}
,{"estado":"Moderado"}
,{"estado":"N/A"}]
```



Contagion types


**source contagion type**
[request](https://datos.gov.co/resource/gt2j-8ykr.json?$select=%20fuente_tipo_contagio&$group=fuente_tipo_contagio)

Json
>```
[
{"fuente_tipo_contagio":"Comunitaria"}
,{"fuente_tipo_contagio":"En estudio"}
,{"fuente_tipo_contagio":"Importado"}
,{"fuente_tipo_contagio":"Relacionado"}
]
```


#Bayesian Algoritm

In [ ]:
#Clustering


Advising by Ian Tomas, after meeting with Michael M. and Sebastian Romero L.

>A good place to start would be a similarity measure in your data, which you can implement with a radial basis function. This function inputs two records from your data set and outputs a single number between 0 and 1 which represents how similar the two records are, where a value closer to 1 means they are more similar, closer to 0 means they are different. This is a good idea to use if you want to represent your data as a graph as typically the similarity measure can work as an edge weight in a graph.

```
 void vectorsubtraction(vector<float> v1, vector<float> v2, vector<float> &v3){

    int i;
    //vectors must be same size
    if(v1.size() != v2.size()) return;
    for(i = 0; i < v1.size(); i++){
        //if symbolic attribute, push 1 to represent not equal, 0 to represent equal
        if(symbolic(i)){
          v3.push_back(v1[i] != v2[i]);
          //if continous, subtract values
        } else {
          v3.push_back((v1[i] - v2[i]));
        }
    }
}

float normalizevector(vector<float> v){

    int i;
    float total = 0.0;
    for(i = 0; i < v.size(); i++){
        total += (v[i] * v[i]);
    }

    return sqrt(total);
}

double edgeweight(vector<float> v1, vector<float> v2){

    vector<float> v3;
    vectorsubtraction(v1, v2, v3);

    float normalized = normalizevector(v3);
    double base = exp(normalized);
    return (1 / base);
}
```



In [ ]:
#Python code of barycentric clustering


In [ ]:
#from sklearn.preprocessing import OneHotEncoder

In [ ]:
#df_to_use_in_pca = df_to_use_in_pca.apply(pd.to_numeric)

#knn

> Step 1: Determine the value for K

> Step 2: Calculate the distances between the new input (test data) and all the training data. The most commonly used metrics for calculating distance are Euclidean, Manhattan and Minkowski

> Step 3: Sort the distance and determine k nearest neighbors based on minimum distance values

> Step 4: Analyze the category of those neighbors and assign the category for the test data based on majority vote

> Step 5: Return the predicted class

#Clustering

In [ ]:
dfn['recuperated']

0          RECUPERADO
1          RECUPERADO
2          RECUPERADO
3          RECUPERADO
4          RECUPERADO
              ...    
4530605        ACTIVO
4530606        ACTIVO
4530607        ACTIVO
4530608        ACTIVO
4530609        ACTIVO
Name: recuperated, Length: 4530610, dtype: string

In [ ]:
#for i in df_to_use_in_pca:
#  df_to_use_in_pca[i].astype(int)

enc = OneHotEncoder(handle_unknown='ignore')

enc_df = pd.DataFrame(
    enc.fit_transform(
        dfn['recuperated']).toarray())

# converted = df_to_use_in_pca.to_numpy()
# print(converted)
# enc.fit(converted)

ValueError: ignored

#Label Encoding

In [ ]:
#from sklearn.preprocessing import LabelEncoder

In [ ]:
df.columns

In [ ]:
df['case_location_encoded'] = LabelEncoder().fit_transform(df['case_location'])
df[['case_location', 'case_location_encoded']] # special syntax to get just these two columns

TypeError: ignored

In [ ]:
df.head(200)

In [ ]:
print(df.dtypes)

Statistics # Statistics to develop